In [1]:
import pandas as pd
df_raw = pd.read_csv('/home/yuqi/callm_multimodality/test/data_additional_exp.csv')

In [6]:
df_raw.columns

Index(['Unnamed: 0', 'Category', 'Question', 'Consistency', 'raw_cot',
       'corrected_cot', 'Correct_Answer', 'Cot_random_answer', 'Smv_Answer',
       'Output_Answer', 'Corrected COT Answer', 'MAD_Answer', 'Hallu Seq'],
      dtype='object')

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
model = ChatOpenAI(api_key='sk-None-gyvgud0r0HNGFPJnYbz5T3BlbkFJZ5liBnFBWxTeyB55sMIY',model="gpt-4o-mini")

system_template = '''
Question:\nAlice buys 5 apples. Each apple costs her $2. She gets them for 50% off. After she bought She decides she doesn't like 2 of them and sells them to her friends for 1. How much money was she out? \n\nAnswer:\nFirst, let's write down all the statements and relationships in the question with labels.\n#1. Alice gets 5 apples.\n#2. Each apple costs $2, but Alice gets apples for 50% off.\n#3. Alice sells 2 of 5 apples for 1.\n#4. How much money was Alice out?\n\nNext, let's answer the question step by step with reference to the question and reasoning process:\n\n#5. (by #2) Step 1: Calculate the price per apple after a 50% discount.\nOriginal price per apple: $2\nDiscount: 50%\nPrice per apple after discount: $2 * (1 - 0.50) = $2 * 0.50 = $1\n\n#6. (by#1 #5) Step 2: Calculate Alice's cost of the 5 apples after a 50% discount.\nPrice per apple after discount: $1\nTotal apples: 5\nAlice's total cost of 5 apples: $1 * 5 = $5\n\n#7. (by #3) Step 3: Calculate the money Alice gets back from selling 2 apples. \nMoney that Alice gets back from selling 2 apples: $1\n\n#8. (by #6 #7) Step 4: Calculate the total amount of money Alice is out. \nAlice's total cost of 5 apples: $5\nMoney that Alice gets back from selling 2 apples: $1\nMoney that Alice is out: $5 - $1 = $4\n\n#9. (by #4 #8) The original question is #4. How much money was Alice out? We do not miss information on the rewritten labels. So the answer to this question is Alice is out $4.\n\nPlease answer the following question by following the above template. 
'''
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "\n\nQuestion:\n{question}\n\nAnswer:\n")]
)

chain = prompt_template | model 

"First, let's write down all the statements and relationships in the question with labels.\n#1. a is a working partner and b is a sleeping partner in a business.\n#2. a puts in $20,000 and b puts in $90,000.\n#3. a gets 20% of the profit for managing the business.\n#4. The rest of the profit is divided in proportion to their capitals.\n#5. The total profit is $11,000.\n#6. Find the share of b in the profit.\n\nNext, let's answer the question step by step with reference to the question and reasoning process:\n\n#7. (by #3) Step 1: Calculate a's share of the profit for managing the business.\nTotal profit: $11,000\na's share for managing: 20% of $11,000 = 0.20 * $11,000 = $2,200\n\n#8. (by #4) Step 2: Calculate the remaining profit after a's share.\nRemaining profit after a's share: $11,000 - $2,200 = $8,800\n\n#9. (by #2) Step 3: Calculate the total capital invested by a and b.\nTotal capital: a's investment + b's investment = $20,000 + $90,000 = $110,000\n\n#10. (by #2) Step 4: Calcula

In [11]:
from tqdm import tqdm
np_prompt = {
    'question': [],
    'answer': [],
    'final_answer': [],
    'example_idx': [],
    'model_input': [],
    'model_output': []
}

for row_idx, row in tqdm(df_raw.iterrows()):
    np_prompt['question'].append(row['Question'])
    np_prompt['answer'].append(row['Correct_Answer'])
    np_prompt['final_answer'].append(row['Correct_Answer'])
    np_prompt['example_idx'].append(row_idx)
    np_prompt['model_input'].append(f'''
Question:\nAlice buys 5 apples. Each apple costs her $2. She gets them for 50% off. After she bought She decides she doesn't like 2 of them and sells them to her friends for 1. How much money was she out? \n\nAnswer:\nFirst, let's write down all the statements and relationships in the question with labels.\n#1. Alice gets 5 apples.\n#2. Each apple costs $2, but Alice gets apples for 50% off.\n#3. Alice sells 2 of 5 apples for 1.\n#4. How much money was Alice out?\n\nNext, let's answer the question step by step with reference to the question and reasoning process:\n\n#5. (by #2) Step 1: Calculate the price per apple after a 50% discount.\nOriginal price per apple: $2\nDiscount: 50%\nPrice per apple after discount: $2 * (1 - 0.50) = $2 * 0.50 = $1\n\n#6. (by#1 #5) Step 2: Calculate Alice's cost of the 5 apples after a 50% discount.\nPrice per apple after discount: $1\nTotal apples: 5\nAlice's total cost of 5 apples: $1 * 5 = $5\n\n#7. (by #3) Step 3: Calculate the money Alice gets back from selling 2 apples. \nMoney that Alice gets back from selling 2 apples: $1\n\n#8. (by #6 #7) Step 4: Calculate the total amount of money Alice is out. \nAlice's total cost of 5 apples: $5\nMoney that Alice gets back from selling 2 apples: $1\nMoney that Alice is out: $5 - $1 = $4\n\n#9. (by #4 #8) The original question is #4. How much money was Alice out? We do not miss information on the rewritten labels. So the answer to this question is Alice is out $4.\n\nPlease answer the following question by following the above template. \n\nQuestion:\n{row['Question']}\n\nAnswer:\n   
''')
    temp_li = []
    for i in range(10):
        temp_li.append(chain.invoke({"question":row['Question']}))
    np_prompt['model_output'].append(temp_li)

290it [3:09:15, 39.16s/it]


In [15]:
sample = np_prompt['model_output'][0]

In [25]:
deve_Answer = []
deve_rp = []
for sample in np_prompt['model_output']:
    deve_rp.append(sample[0].content)
    ans = sample[0].content.split('the answer')[-1]


    deve_Answer.append(parse_answer(ans))
np_prompt['deve_Answer'] = deve_Answer

df_deve = pd.DataFrame({'deve_rp':deve_rp, 'deve_Answer':deve_Answer})
df_deve.to_csv('deve.csv', index=False)

In [23]:
import re 
def parse_answer(answer):
    pattern = r'[a-z]\)'
    match = re.search(pattern, answer)
    if match:
        return match.group()[0]
    else:
        return answer